In [1]:
#Download Dataset

import downloader



downloader.get_drinks_dataset()

downloader.get_model()

Directory: data\ - exists
File: 		data\drinks.tar.gz
File Size:	146820533
Data Set Already Downloaded
Drinks Folder Already Exists
File: 		proj2_model.pth
File Size:	178134939
Data Set Already Downloaded


True

In [2]:
# import os
# #Get Model from Git
# pret_model_url = 'https://github.com/atsantiago2/dl_course_proj2.git'
# pret_model_dirname = 'git_pretrained_models'
# pret_model_filename = 'proj2_model.pth'
# pret_model_dirpath = os.path.join(pret_model_dirname, '')
# pret_model_path = os.path.join(pret_model_dirname, pret_model_filename)

# #Download Model from Online
# from git import Repo
# if os.path.isdir(pret_model_dirpath):
# 	print('Pretrained Model Already Downloaded Before')
# else:
# 	Repo.clone_from(pret_model_url, pret_model_dirpath)
# 	print('Pretrained Model Downloaded Already')


# #Copy Model to local
# import shutil
# if os.path.isfile(pret_model_filename):
# 	print('Pretrained Model Already in Place')
# else:
# 	shutil.copyfile(pret_model_path, pret_model_filename)
# 	print('Pretrained Model Copy Complete')


In [3]:
#Get Pretrained Model to use
# import torchvision
# from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
# from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
# model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

In [4]:
# import torch
# torch.save(model.state_dict(), pret_model_filename)

In [5]:
config = {
  "learning_rate": 0.1,
  "epochs": 10,
  "batch_size": 128,
  "dataset": "drinks"
}

In [6]:
import os
# import label_utils
import dataload
data_dir = 'data'
test_csv_path = os.path.join(data_dir, 'drinks', 'labels_test.csv')
# test_dict, test_classes = label_utils.build_label_dictionary(test_csv_path)
test_dict, test_classes = dataload.get_dataset_dict(train=False)

test_csv_path = os.path.join(data_dir, 'drinks', 'labels_train.csv')
train_dict, train_classes = dataload.get_dataset_dict(train=True)


# test_seg_dict, test_seg_classes =
train_dict['data\\drinks\\0000000.jpg']

Total Unique Masks 3
Unique Masks ['1', '2', '3']
Total Unique Masks 3
Unique Masks ['3', '1', '2']


{'filename': 'data\\drinks\\0000000.jpg',
 'masks': [{'Name': '3',
   'x': [367, 393, 415, 430, 433, 418, 403, 385, 369, 355],
   'y': [305, 301, 304, 311, 315, 467, 471, 472, 468, 461]},
  {'Name': '1',
   'x': [531, 555, 568, 571, 569, 567, 581, 559, 549, 489, 479, 505, 527],
   'y': [220, 220, 226, 229, 243, 251, 275, 473, 479, 478, 468, 261, 245]},
  {'Name': '2', 'x': [631, 639, 639, 605], 'y': [313, 311, 475, 475]}]}

In [7]:
# import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image

class Res50Dataset(torch.utils.data.Dataset):
	def __init__(self, dictionary, root, transforms=None):
		self.dictionary = dictionary
		self.root = root
		self.transforms = transforms
		# load all image files, sorting them to
		# ensure that they are aligned
		# self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
		# self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

	def __getitem__(self, idx):
		key = list(self.dictionary.keys())[idx]
		d_val = self.dictionary[key]
		# print('k', key)
		# print('d',d_val)


		# load images ad masks
		img_path = d_val['filename']
		img = Image.open(img_path).convert("RGB")
		# note that we haven't converted the mask to RGB,
		# because each color corresponds to a different instance
		# with 0 being background

		masks = d_val['masks']
		# print('m',masks)
		# mask = Image.open(mask_path)
		# mask = np.array(mask)
		# instances are encoded as different colors
		# obj_ids = np.unique(mask)
		# first id is the background, so remove it
		# obj_ids = obj_ids[1:]

		# split the color-encoded mask into a set
		# of binary masks

		num_objs = len(masks)
		# get bounding box coordinates for each mask
		mask_list = []
		boxes = []
		areas = []
		labels = []
		for mask in masks:
			name = int(mask['Name'])
			mask_list.append(name)
			if name not in labels:
				labels.append(name)

			xmin = np.min(mask['x'])
			xmax = np.max(mask['x'])
			ymin = np.min(mask['y'])
			ymax = np.max(mask['y'])
			boxes.append([xmin, ymin, xmax, ymax])
			
			areas.append((ymax - ymin) * (xmax - xmin))

		# obj_ids = labels
		boxes = torch.as_tensor(boxes, dtype=torch.float32)

		# there is only one class
		labels = torch.as_tensor(labels, dtype=torch.int64)
		area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

		print('ZZZ:', mask_list)
		# masks = torch.as_tensor(masks, dtype=torch.uint8)

		id = key[key.find(self.root) + len(self.root) + 1:key.rfind('.jpg')]
		id = int(id)
		# print(id)
		image_id = torch.tensor([id])
		# suppose all instances are not crowd
		iscrowd = torch.zeros(labels.size(), dtype=torch.int64)

		target = {}
		target["boxes"] = boxes		#each mask
		target["labels"] = labels	# Each bbox
		# target["masks"] = [masks]
		target["image_id"] = image_id
		target["area"] = area
		target["iscrowd"] = iscrowd

		if self.transforms is not None:
			img, target = self.transforms(img, target)

		return img, target

	def __len__(self):
		return len(self.dictionary)


# train_dataset = Res50Dataset(
# 						dictionary = train_dict,
# 						root = os.path.join(data_dir, 'drinks'),
# 						)
# train_dataset.__getitem__(1)

C:\ProgramData\Anaconda3\envs\dl2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:

# train_dataset = Res50Dataset(
# 						dictionary = train_dict,
# 						root = os.path.join(data_dir, 'drinks'),
# 						)
# train_dataset.__getitem__(1)

In [9]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [10]:

model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

# get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
in_features

1024

In [11]:
model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [12]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Ten|sor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [13]:
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# dataset = Res50Dataset(
# 						dictionary = train_dict,
# 						root = os.path.join(data_dir, 'drinks'),
# 						)
# data_loader = torch.utils.data.DataLoader(
#     dataset, batch_size=2, shuffle=True, num_workers=4,
#     collate_fn=utils.collate_fn
# )
# # For Training
# images,targets = next(iter(data_loader))
# images = list(image for image in images)
# targets = [{k: v for k, v in t.items()} for t in targets]
# output = model(images,targets)   # Returns losses and detections
# # For inference
# model.eval()
# x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# predictions = model(x)           # Returns predictions


In [14]:

# #Get Model from Git
# vision_git_url = 'https://github.com/pytorch/vision.git'
# vision_git_dirname = 'vision'
# vision_git_filename = 'testfile.txt'
# vision_git_dirpath = os.path.join(vision_git_dirname, '')
# vision_git_path = os.path.join(vision_git_dirname, vision_git_filename)

# #Download Model from Online
# from git import Repo
# if os.path.isdir(vision_git_dirpath):
# 	print('Pretrained Model Already Downloaded Before')
# else:
# 	Repo.clone_from(vision_git_url, vision_git_dirpath)
# 	print('Pretrained Model Downloaded Already')

	
# # Download TorchVision repo to use some files from
# # references/detection

# # cp references/detection/utils.py ../
# # cp references/detection/transforms.py ../
# # cp references/detection/coco_eval.py ../
# # cp references/detection/engine.py ../
# # cp references/detection/coco_utils.py ../

In [15]:
# use our dataset and defined transformations
dataset_train = Res50Dataset(
						dictionary = train_dict,
						root = os.path.join(data_dir, 'drinks'),
						transforms=get_transform(train=True)
						)
dataset_test = Res50Dataset(
						dictionary = test_dict,
						root = os.path.join(data_dir, 'drinks'),
						transforms=get_transform(train=False)
						)

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset_train)).tolist()
dataset_train = torch.utils.data.Subset(dataset_train, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=100, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=100, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [16]:
i,a = data_loader.dataset.__getitem__(4)
a

ZZZ: [2, 3]


{'boxes': tensor([[327.,  76., 425., 323.],
         [337., 315., 424., 479.]]),
 'labels': tensor([2, 3]),
 'image_id': tensor([239]),
 'area': tensor([24206., 14268.]),
 'iscrowd': tensor([0, 0])}

In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 4

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [18]:
device

device(type='cuda')

In [19]:
# let's train it for 10 epochs
from torch.optim.lr_scheduler import StepLR
num_epochs = 1

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=1)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

Train Start
1
2
3
4
WarmUp Done
